In [1]:
# --- 1. Install dependencies ---
!pip install -q transformers accelerate peft bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 15.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login("GIVE YOUR TOKEN HERE")

In [4]:
from huggingface_hub import create_repo

create_repo("mistral-7b-ipc-merged", private=False)  # or private=True if you want it private


RepoUrl('https://huggingface.co/arjun-ms/mistral-7b-ipc-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='arjun-ms/mistral-7b-ipc-merged')

In [5]:
# --- 3. Imports ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from huggingface_hub import HfApi

# --- 4. Model IDs ---
base_id = "mistralai/Mistral-7B-Instruct-v0.2"  # base Mistral (make sure you have access!)
lora_id = "arjun-ms/mistral-lora-ipc"           # your LoRA adapter
merged_id = "./mistral-ipc-lora-merged"         # local save folder
push_repo_id = "arjun-ms/mistral-7b-ipc-merged" # NEW HF repo name

# --- Load base in full precision, no offload ---
base_model = AutoModelForCausalLM.from_pretrained(
    base_id,
    torch_dtype=torch.float16,  # or float32 if enough RAM
    low_cpu_mem_usage=False,    # keep full layer structure in memory
    device_map=None             # don't offload yet
)

# --- Attach LoRA ---
model = PeftModel.from_pretrained(
    base_model,
    lora_id
)

# --- Merge ---
model = model.merge_and_unload()

# --- Save merged model ---
model.save_pretrained(merged_id, safe_serialization=True)

# --- Save tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(base_id)
tokenizer.save_pretrained(merged_id)

# --- 10. Push to Hugging Face ---
api = HfApi()
api.upload_folder(
    folder_path=merged_id,
    repo_id=push_repo_id,
    repo_type="model"
)

print(f"✅ Merged model pushed to: https://huggingface.co/{push_repo_id}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

✅ Merged model pushed to: https://huggingface.co/arjun-ms/mistral-7b-ipc-merged
